In [6]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("..")

from data_processing.data_generate import generate_random_matrix

## 1.逻辑回归（Logistic Regression）公式

逻辑回归模型的输出可以表示为：

$$
f(x) = \frac{1}{1 + e^{-(w \cdot x + b)}}
$$

在使用梯度下降法来优化逻辑回归模型时，第一步是明确逻辑回归的目标函数。根据上述公式，逻辑回归的数学形式为 $f_w(x)$。对于一个输入样本 $x$，其预测结果可以是正样本（类别 1）或负样本（类别 0），对应的概率分别为：

$$
\begin{cases}
P(y = 1 | x; w) = f_w(x) \\
P(y = 0 | x; w) = 1 - f_w(x)
\end{cases}
$$

将其联合起来，得到数据点的条件概率表达式：

$$
p(y | x; w) = (f_w(x))^y(1 - f_w(x))^{1-y}
$$

**根据极大似然估计的原理，逻辑回归的目标函数可以写成以下形式**。我们的目标是通过优化模型参数 $w$，最大化给定训练数据集的似然函数。换句话说，我们希望通过学习模型参数，使得在该模型下观察到的样本数据出现的概率最大化。

$$
L(w) = \prod_{i=1}^{m} P(y_i | x_i; w)
$$


### 1.1目标函数

逻辑回归的目标函数初始是一个连乘形式，直接对其求导较为复杂。因此，我们通常**对目标函数两侧取对数**，并**乘以系数 $-\frac{1}{m}$**，这样可以将原本的最大化问题转化为最小化问题。通过取对数，连乘操作变为连加，最终得到的目标函数形式如下：

$$
\begin{aligned}
J(w) &= -\frac{1}{m}l(w) = -\frac{1}{m}L(w) \\
     &= -\frac{1}{m} \sum_{i=1}^{m} \left( y^i \log f_w(x^i) + (1 - y^i) \log (1 - f_w(x^i)) \right)
\end{aligned}
$$

在得到逻辑回归的目标函数后，接下来的步骤是对每个参数进行偏导数计算，以确定其梯度方向。具体来说，对于目标函数 $J(w)$ 中的参数 $w_j$，我们求其偏导数，结果如下：

$$
\frac{\partial J(w)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} \left( f_w(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

其中：

- $y^{(i)}$ 代表第 $i$ 个样本的真实标签。

- $w_j$ 是逻辑回归模型的参数，而 $x_j^{(i)}$ 则是第 $i$ 个样本的第 $j$ 个特征。

损失函数对偏置 b 的偏导数计算如下：

$$
\frac{\partial L(w, b)}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} \left( f_w(x^{(i)}) - y^{(i)} \right)
$$

其中：

- $f_w(x^{(i)})$ 是第 i 个样本的预测值，计算公式为：  
  
- $y^{(i)}$ 是第 i 个样本的真实标签。

在得到梯度之后，我们利用梯度下降法来更新模型参数。梯度下降的更新规则为：

$$
w_j := w_j - \alpha \cdot \frac{1}{m} \sum_{i=1}^{m} \left( h_w(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

$$
b = b - \alpha \cdot  \frac{1}{m} \sum_{i=1}^{m} \left( f_w(x^{(i)}) - y^{(i)} \right)
$$

通过以上步骤，我们就完成了逻辑回归模型参数更新的推导过程。


### 2 代码实现

In [57]:
def sigmoid(w,x,b):
    """
    计算sigmoid函数
    Args:
      w (ndarray): 权重参数,n*1
      x (ndarray): 样本矩阵,m*n
      b (scalar): 偏置项
    Returns:
      p (ndarray): sigmoid计算结果,m*1
    """
    
    z = np.dot(x,w) + b
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b, fw = None):
    """
    计算损失函数
    Args:
      X (ndarray): 样本矩阵,m*n
      y (ndarray): 标签矩阵,m*1
      w (ndarray): 权重参数,n*1
      b (scalar): 偏置项
    Returns:
      cost (scalar): 损失函数值
    """
    m = X.shape[0]
    #同样使用矩阵计算

    #如果已经计算了fw,就不用再计算一遍了sigmoid
    if fw is not None :
      cost = -(1/m) *  np.sum((y * np.log(fw) + (1 - y) * np.log(1 - fw)))
    else:
      cost = -(1/m) *  np.sum((y * np.log(sigmoid(w, X, b)) + (1 - y) * np.log(1 - sigmoid(w, X, b))))
    return cost


In [58]:
def logicstic_regression(X, y, learning_rate = 0.01, num_iterations = 1000):
    """
    X: input data
    y: output data
    learning_rate: learning rate for gradient descent
    num_iterations: number of iterations for gradient descent
    return: weights and bias
    """
    
    rows,cols = X.shape
    w = np.random.rand(cols,1)
    b = 0 
    
    pbar = tqdm(range(num_iterations),desc ='Logicstic Regression',ncols = 100,unit = 'it')

    for i in pbar:
        predictions = sigmoid(w,X,b).astype(np.float32)
        cost = compute_cost(X, y, w, b,fw = predictions)
        
        # 注意这里用矩阵计算,纬度是m*1
        error = predictions - y 
        dw = (1/rows) * np.dot(X.T,error) 
        db = (1/rows) * np.sum(error)

        w -= learning_rate * dw
        b -= learning_rate * db
        
        pbar.set_postfix({"cost": cost})

    return w, b


In [59]:
rows,cols = 4,5
X = generate_random_matrix(rows = rows, cols = cols, min_value = 1, max_value = 5)
y = np.random.randint(0,2, size = (rows,1))


w,b = logicstic_regression(X = X, y = y,learning_rate = 0.01, num_iterations = 1000 )



Logicstic Regression: 100%|██████████████████████| 1000/1000 [00:00<00:00, 1190.12it/s, cost=0.0357]


In [63]:
#初始矩阵
prediction = sigmoid(w,X,b)
df = pd.DataFrame(np.concatenate((X,y), axis = 1),columns = [f'x{i}' for i in range(X.shape[1])] + ['y'])

df['y_pred'] = prediction

df

,x0,x1,x2,x3,x4,y,y_pred
0,3,4,4,5,2,0,0.054166
1,2,3,1,5,5,0,0.000324
2,1,1,3,1,5,0,0.020765
3,5,4,5,2,2,1,0.936527
